API Documentary Source: https://wiki.opencellid.org/wiki/API

The link above provides a description of the columns that can be found in the API response data

MCC Country Codes can be found in the following website: https://en.wikipedia.org/wiki/Mobile_country_code

The first digit of the mobile country code identifies the geographic region as follows (the digits 1 and 8 are not used):
- 0: Test networks
- 2: Europe
- 3: North America and the Caribbean
- 4: Asia and the Middle East
- 5: Australia and Oceania
- 6: Africa
- 7: South and Central America
- 9: Worldwide (Satellite, Air—aboard aircraft, Maritime—aboard ships, Antarctica)

In [66]:
import pandas as pd
import math
import numpy as np
import json
from geopy.distance import distance
from keplergl import KeplerGl

Read the data and config

In [67]:
data = pd.read_csv('Data/cell_towers.csv')

In [68]:
with open('config.json', 'r') as f:
  config = json.load(f)

Drop Irrelevant Columns

In [69]:
del data['unit']
del data['changeable']
del data['created']
del data['updated']
del data['averageSignal']
del data['net']
del data['samples']

Random Location

In [70]:
# data['mcc'] = data['mcc'].astype(str)
# subset = data[data['mcc'].str.match(f"^{continents['Asia and the Middle East']}")]
# subset = data[data.mcc.isin([457,502,515,520,452,528,414,525,456,510])].reset_index(drop=True)
subset = data[data.mcc.isin([250])].reset_index(drop=True)
lte = subset[subset.radio == 'LTE'].reset_index(drop = True)
umts = subset[subset.radio == 'UMTS'].reset_index(drop = True)
gsm = subset[subset.radio == 'GSM'].reset_index(drop = True)

Modify config

In [71]:
config['config']['mapState']['latitude'] = subset['lat'].mean()
config['config']['mapState']['longitude'] = subset['lon'].mean()

# Find the minimum and maximum latitude and longitude
min_lat, min_lon = subset['lat'].min(), subset['lon'].min()
max_lat, max_lon = subset['lat'].max(), subset['lon'].max()

# Calculate the distance between the top-left and bottom-right corners of the bounding box
distance_m = distance((max_lat, min_lon), (min_lat, max_lon)).meters

# Set the desired width or height of the map in pixels
width_px = 800
height_px = 600

# Calculate the zoom level based on the distance and desired size of the map
config['config']['mapState']['zoom'] = math.floor(math.log2((40075016.686 * width_px) / (distance_m * 256)))

Create the visual on Kepler.gl

In [72]:
cell_tower_visual = KeplerGl(height=800, data={"LTE Cell Towers": lte}, config=config)
cell_tower_visual.add_data(data=umts, name='UMTS Cell Towers')
cell_tower_visual.add_data(data=gsm, name='GSM Cell Towers')
cell_tower_visual

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': '4e3poy', 'type': '…

In [24]:
cell_tower_visual.save_to_html(file_name='Cell Towers - Russia.html')

Map saved to Cell Towers - Russia.html!
